# Discrete Anomaly Detection

Use basic probability to identify anomalous requests. Using the methods covered in the lesson, examine the rest of the features in the api access logs data set.

In [2]:
import numpy as np
import pandas as pd
import env

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# function to deal with parsing one entry in our log data
def parse_log_entry(entry):
    parts = entry.split()
    output = {}
    output['ip'] = parts[0]
    output['timestamp'] = parts[3][1:].replace(':', ' ', 1)
    output['request_method'] = parts[5][1:]
    output['request_path'] = parts[6]
    output['http_version'] = parts[7][:-1]
    output['status_code'] = parts[8]
    output['size'] = int(parts[9])
    output['user_agent'] = ' '.join(parts[11:]).replace('"', '')
    return pd.Series(output)

url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/logs'
df = pd.read_sql('SELECT * FROM api_access', url)
# df = pd.concat([df.entry, df.entry.apply(parse_log_entry)], axis=1)
df = df.entry.apply(parse_log_entry)
df.head()

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,16/Apr/2019 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,16/Apr/2019 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,16/Apr/2019 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0


In [4]:
# Synthetic data
new = pd.DataFrame([
    ["95.31.18.119", "21/Apr/2019 10:02:41", "GET", "/api/v1/items/", "HTTP/1.1", '200', 1153005, "python-requests/2.21.0"],
    ["95.31.16.121", "17/Apr/2019 19:36:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '301', 1005, "python-requests/2.21.0"],
    ["97.105.15.120", "18/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '301', 2560, "python-requests/2.21.0"],
    ["97.105.19.58", "19/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '200', 2056327, "python-requests/2.21.0"],
], columns=df.columns)

df = df.append(new)

In [5]:
df['size_mb'] = df['size'] / 1024 / 1024
df.timestamp = pd.to_datetime(df.timestamp)
df = df.set_index('timestamp')

In [6]:
def value_counts_and_frequencies(s: pd.Series, dropna=True) -> pd.DataFrame:
    return pd.merge(
        s.value_counts(dropna=False).rename('count'),
        s.value_counts(dropna=False, normalize=True).rename('proba'),
        left_index=True,
        right_index=True,
    )

In [7]:
request_methods = value_counts_and_frequencies(df.request_method)

In [8]:
request_methods

,count,proba
GET,13978,1.0


In [9]:
https_versions = value_counts_and_frequencies(df.http_version)

In [10]:
https_versions

,count,proba
HTTP/1.1,13978,1.0


In [11]:
user_agents = value_counts_and_frequencies(df.user_agent)

In [12]:
user_agents

,count,proba
python-requests/2.21.0,12005,0.858850
python-requests/2.20.1,1911,0.136715
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36",34,0.002432
Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0,8,0.000572
Slackbot-LinkExpanding 1.0 (+https://api.slack.com/robots),7,0.000501
Slackbot 1.0 (+https://api.slack.com/robots),6,0.000429
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36",4,0.000286
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36",2,0.000143
Python-urllib/3.7,1,0.000072


The majority of user_agents were 'python-requests/2.21.0', followed by 'python-requests/2.20.1'. The others all seem like anomalies. Are those user_agents legitimate? I can check the times those requests were made to see if anything stands out.

In [13]:
df[df.user_agent == 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36']

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb
timestamp,,,,,,,,
2019-04-16 19:38:16,97.105.19.58,GET,/api/V1/HiZach!,HTTP/1.1,200,162,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.000154
2019-04-16 19:38:16,97.105.19.58,GET,/favicon.ico,HTTP/1.1,200,162,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.000154
2019-04-16 19:44:04,97.105.19.58,GET,/api/v1/stores?page=0,HTTP/1.1,200,1328,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.001266
2019-04-16 19:44:04,97.105.19.58,GET,/favicon.ico,HTTP/1.1,200,162,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.000154
2019-04-16 19:44:16,97.105.19.58,GET,/api/v1/stores?page=1,HTTP/1.1,200,1328,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.001266
2019-04-16 19:44:16,97.105.19.58,GET,/favicon.ico,HTTP/1.1,200,162,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.000154
2019-04-16 19:44:24,97.105.19.58,GET,/api/v1/stores?page=1,HTTP/1.1,200,1328,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.001266
2019-04-16 19:44:24,97.105.19.58,GET,/favicon.ico,HTTP/1.1,200,162,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.000154
2019-04-16 19:44:28,97.105.19.58,GET,/api/v1/stores?page=2,HTTP/1.1,200,1328,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.001266


There are only two ip addresses associated with this user_agent. While there are some interesting request_paths, nothing seems really out of the ordinary.

In [14]:
df[df.user_agent == 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0']

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb
timestamp,,,,,,,,
2019-04-17 01:25:07,72.181.105.81,GET,/,HTTP/1.1,200,42,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,0.000040
2019-04-17 01:25:21,72.181.105.81,GET,/documentation,HTTP/1.1,200,348,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,0.000332
2019-04-17 01:33:06,72.181.105.81,GET,/api/v1/items?page=2,HTTP/1.1,200,3659,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,0.003489
2019-04-17 01:33:19,72.181.105.81,GET,/api/v1/items?page=3,HTTP/1.1,200,1787,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,0.001704
2019-04-17 01:33:32,72.181.105.81,GET,/api/v1/items?page=4,HTTP/1.1,200,48,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,0.000046
2019-04-17 01:56:33,72.181.105.81,GET,/api/v1/stores,HTTP/1.1,200,1328,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,0.001266
2019-04-17 01:56:57,72.181.105.81,GET,/api/v1/sales,HTTP/1.1,200,493905,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,0.471025
2019-04-17 02:07:44,72.181.105.81,GET,/api/v1/sales,HTTP/1.1,200,493905,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,0.471025


There is only one ip address associated with this user_agent and it is one of the ip addresses from the previous user_agent. Since the user_agent is different, I'm assuming this is a different machine utilizing the same network as the other one.

In [15]:
df[df.user_agent == 'Slackbot-LinkExpanding 1.0 (+https://api.slack.com/robots)']

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb
timestamp,,,,,,,,
2019-04-16 19:39:14,3.92.201.136,GET,/,HTTP/1.1,200,42,Slackbot-LinkExpanding 1.0 (+https://api.slack...,0.000040
2019-04-16 19:40:32,3.88.129.158,GET,/api/v1/items,HTTP/1.1,200,3561,Slackbot-LinkExpanding 1.0 (+https://api.slack...,0.003396
2019-04-16 19:40:36,54.172.14.223,GET,/api/v1/,HTTP/1.1,200,162,Slackbot-LinkExpanding 1.0 (+https://api.slack...,0.000154
2019-04-16 21:22:22,35.175.171.137,GET,/,HTTP/1.1,200,42,Slackbot-LinkExpanding 1.0 (+https://api.slack...,0.000040
2019-04-16 21:22:23,52.87.230.102,GET,/api/v1/sales,HTTP/1.1,200,493905,Slackbot-LinkExpanding 1.0 (+https://api.slack...,0.471025
2019-04-16 21:38:57,35.175.171.137,GET,/api/v1/items,HTTP/1.1,200,3561,Slackbot-LinkExpanding 1.0 (+https://api.slack...,0.003396
2019-04-17 02:14:27,52.91.30.150,GET,/,HTTP/1.1,200,42,Slackbot-LinkExpanding 1.0 (+https://api.slack...,0.000040


Okay, I'm moving on to look at the different request paths.

In [16]:
request_paths = value_counts_and_frequencies(df.request_path)

In [17]:
request_paths

,count,proba
/api/v1/sales?page=2,709,0.050723
/api/v1/items,464,0.033195
/api/v1/items?page=2,291,0.020818
/api/v1/items?page=3,219,0.015667
/api/v1/stores,162,0.011590
...,...,...
/api/v1,1,0.000072
/api/v1//api/v1/items/next_page,1,0.000072
/api/v1//api/v1/items,1,0.000072
/api/v1/items?page=0,1,0.000072


There are definitely some request paths that are not related to the items, stores, or sales pages.